### Levende rutenett
  
Hvordan i all verden kan noe så komplekst som *liv* oppstå ut fra forholdsvis enkle regler og lover fra kjemi og fysikk?

Hvorfor i all verden snakker jeg om dette i et kapittel for faget Informasjonsteknologi 2?

Alt vil bli klart, kjære leser, når vi tar et dypdykk i **Conway's Game of Life**...


#### Game of Life

**Game of Life** er et sett med regler for et rutenett som ble utviklet av John Horton Conway på 70-tallet. Hver rute i rutenettet anses som en *celle*, og hver celle kan være enten levende eller død. Reglene er som følger:

Naboer | Oppdatering
---|---
Færre enn to levende naboer | Dør
To eller tre levende naboer | Fortsetter å leve
Flere enn tre levende naboer | Dør
Død, men med akkurat tre levende naboer | Kommer til live

```{admonition} Oppgave: Lag Game of Life
:class: task

Under kommer et forslag på en løsning for Game of Life, men prøv gjerne selv først!
```

Her er 